<a href="https://colab.research.google.com/github/OscarVC2/SE_hackathon_2022/blob/main/2022_05_21_SE_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget "https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train1.csv"
!wget "https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train2.csv"
!wget "https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train6.zip"
!unzip train6.zip

--2022-06-11 06:05:44--  https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train1.csv
Resolving challenges-asset-files.s3.us-east-2.amazonaws.com (challenges-asset-files.s3.us-east-2.amazonaws.com)... 52.219.92.122
Connecting to challenges-asset-files.s3.us-east-2.amazonaws.com (challenges-asset-files.s3.us-east-2.amazonaws.com)|52.219.92.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8153873 (7.8M) [text/csv]
Saving to: ‘train1.csv’

train1.csv          100%[===================>]   7.78M  31.4MB/s    in 0.2s    

2022-06-11 06:05:45 (31.4 MB/s) - ‘train1.csv’ saved [8153873/8153873]

--2022-06-11 06:05:45--  https://challenges-asset-files.s3.us-east-2.amazonaws.com/data_sets/Data-Science/4+-+events/SchneiderElectricES22/final/train2.csv
Resolving challenges-asset-files.s3.us-east-2.amazonaws.com (challenges-asset-files.s3.us-east-2.amazonaws.com)... 52.219.92.122
Connecting to challeng

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy.stats
from scipy.stats import iqr
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, auc, f1_score, precision_score, recall_score,\
                            roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix

warnings.filterwarnings("ignore")
pd.set_option('display.max.rows',169)
pd.set_option('display.max.columns',169)
pd.set_option('float_format', '{:.2f}'.format)

### Get data files - csv, json, pdf

In [3]:
train1 = pd.read_csv('train1.csv')

In [4]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18563 entries, 0 to 18562
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   countryName                   18563 non-null  object 
 1   eprtrSectorName               18563 non-null  object 
 2   EPRTRAnnexIMainActivityLabel  18563 non-null  object 
 3   FacilityInspireID             18563 non-null  object 
 4   facilityName                  18563 non-null  object 
 5   City                          18563 non-null  object 
 6   targetRelease                 18563 non-null  object 
 7   pollutant                     18563 non-null  object 
 8   reportingYear                 18563 non-null  int64  
 9   MONTH                         18563 non-null  int64  
 10  DAY                           18563 non-null  int64  
 11  CONTINENT                     18563 non-null  object 
 12  max_wind_speed                18563 non-null  float64
 13  a

In [5]:
true_list = ['countryName',
             'EPRETRSectorCode', 
             'eptrSectorName', 
             'EPRTRAnnexIMainActivityCode', 
             'EPRTRAnnexIMainActivityLabel', 
             'FacilityInspireID', 
             'facilityName', 
             'City', 
             'CITY ID ', 
             'targetRelease', 
             'pollutant', 
             'DAY', 
             'MONTH', 
             'reportingYear', 
             'CONTINENT', 
             'max_wind_speed', 
             'avg_wind_speed', 
             'min_wind_speed', 
             'max_temp', 
             'avg_temp', 
             'min_temp', 
             'DAYS WITH FOG', 
             'REPORTER NAME']

In [6]:
train2 = pd.read_csv("train2.csv", sep=";")

In [7]:
train2.shape

(18564, 21)

In [8]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18564 entries, 0 to 18563
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   countryName                   18564 non-null  object 
 1   eprtrSectorName               18564 non-null  object 
 2   EPRTRAnnexIMainActivityLabel  18564 non-null  object 
 3   FacilityInspireID             18564 non-null  object 
 4   facilityName                  18564 non-null  object 
 5   City                          18564 non-null  object 
 6   targetRelease                 18564 non-null  object 
 7   pollutant                     18564 non-null  object 
 8   reportingYear                 18564 non-null  int64  
 9   MONTH                         18564 non-null  int64  
 10  DAY                           18564 non-null  int64  
 11  CONTINENT                     18564 non-null  object 
 12  max_wind_speed                18564 non-null  float64
 13  a

In [9]:
train1.columns == train2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

### Get the json files via API

In [10]:
import requests
import json

In [11]:
response = requests.get('http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first').text

In [12]:
response_info = json.loads(response)

In [13]:
train3 = pd.DataFrame(response_info)

In [14]:
response2 = requests.get('http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second').text

In [15]:
response_info2 = json.loads(response2)

In [16]:
train4 = pd.DataFrame(response_info2)

In [17]:
response3 = requests.get('http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third').text

In [18]:
response_info3 = json.loads(response3)

In [19]:
train5 = pd.DataFrame(response_info3)

In [20]:
train4.shape

(9500, 24)

In [21]:
train5.shape

(9501, 24)

In [22]:
train3.shape

(9500, 24)

In [23]:
train4.columns == train5.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [24]:
train4.columns == train3.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

Uniendo las siguientes 2 bases de datos

In [25]:
true_list = ['countryName',
             'EPRETRSectorCode', 
             'eptrSectorName', 
             'EPRTRAnnexIMainActivityCode', 
             'EPRTRAnnexIMainActivityLabel', 
             'FacilityInspireID', 
             'facilityName', 
             'City', 
             'CITY ID ', 
             'targetRelease', 
             'pollutant', 
             'DAY', 
             'MONTH', 
             'reportingYear', 
             'CONTINENT', 
             'max_wind_speed', 
             'avg_wind_speed', 
             'min_wind_speed', 
             'max_temp', 
             'avg_temp', 
             'min_temp', 
             'DAYS WITH FOG', 
             'REPORTER NAME']

In [26]:
len(true_list)

23

In [27]:
df1 = pd.concat([train3,train4,train5], axis=0)

In [28]:
df1.drop('', axis=1, inplace=True)

In [29]:
sector_code = df1[['eprtrSectorName', 'EPRTRSectorCode']]

In [30]:
activity_code = df1[['EPRTRAnnexIMainActivityLabel', 'EPRTRAnnexIMainActivityCode']]

In [31]:
df2 = pd.concat([train1,train2],axis=0)

In [32]:
df2 = df2.drop_duplicates(keep='first')

In [33]:
df2.shape

(28559, 21)

In [34]:
#B = df2[:len(df2)//2]
#C = df2[len(df2)//2:]

In [35]:
#B= B.merge(activity_code, on='EPRTRAnnexIMainActivityLabel')

In [36]:
#B = B.merge(sector_code, on='eprtrSectorName')

## Import PDF files

In [37]:
pip install tika

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from tika import parser
from bs4 import BeautifulSoup
import glob
import os
import json
from tqdm import tqdm
import tika
tika.initVM()

In [39]:
path_to_data = "train6/"
doctypes = ['doc', 'docx', 'pdf', 'txt', 'pptx', 'ppt']
pepe = [glob.glob(path_to_data + f'/**/*.{extension}', recursive=True) for extension in doctypes]
list_of_docs = [var for sublist in pepe for var in sublist]
list_of_docs

['train6/pdfs81587.pdf',
 'train6/pdfs81562.pdf',
 'train6/pdfs81584.pdf',
 'train6/pdfs81535.pdf',
 'train6/pdfs81593.pdf',
 'train6/pdfs81553.pdf',
 'train6/pdfs81590.pdf',
 'train6/pdfs81528.pdf',
 'train6/pdfs81574.pdf',
 'train6/pdfs81558.pdf',
 'train6/pdfs81577.pdf',
 'train6/pdfs81578.pdf',
 'train6/pdfs81541.pdf',
 'train6/pdfs81515.pdf',
 'train6/pdfs81548.pdf',
 'train6/pdfs81544.pdf',
 'train6/pdfs81588.pdf',
 'train6/pdfs81589.pdf',
 'train6/pdfs81520.pdf',
 'train6/pdfs81582.pdf',
 'train6/pdfs81580.pdf',
 'train6/pdfs81540.pdf',
 'train6/pdfs81576.pdf',
 'train6/pdfs81531.pdf',
 'train6/pdfs81554.pdf',
 'train6/pdfs81567.pdf',
 'train6/pdfs81549.pdf',
 'train6/pdfs81551.pdf',
 'train6/pdfs81518.pdf',
 'train6/pdfs81581.pdf',
 'train6/pdfs81583.pdf',
 'train6/pdfs81527.pdf',
 'train6/pdfs81523.pdf',
 'train6/pdfs81569.pdf',
 'train6/pdfs81538.pdf',
 'train6/pdfs81547.pdf',
 'train6/pdfs81545.pdf',
 'train6/pdfs81592.pdf',
 'train6/pdfs81568.pdf',
 'train6/pdfs81542.pdf',


In [40]:
def extract_text_from_pdf(file, sep='\t\n\t\n\t\n'):
  raw_xml = parser.from_file(file, xmlContent=True)
  body = raw_xml['content'].split('<body>')[1].split('</body>')[0]
  body_without_tag = body.replace("<p>", "\n").replace("</p>", "").replace("<div>", "").replace("</div>","").replace("<p />","\n")
  text_pages = body_without_tag.split("""<div class="page">""")[1:]
  num_pages = len(text_pages)
  return sep.join(text_pages)


In [41]:
raw_docs = [extract_text_from_pdf(doc) for doc in list_of_docs]
df_docs = pd.DataFrame(columns = ['raw_text'], data = raw_docs)
df_docs.to_csv('pdf_documents_data_6.csv')

In [42]:
raw6 = pd.read_csv('pdf_documents_data_6.csv', index_col = 0)

In [43]:
raw6['raw_text'] = raw6['raw_text'].str.replace('\n\n\nREPORT CONTAMINACIÓN\n\n\n', 'REPORT CONTAMINACIÓN ')
raw6['raw_text'] = raw6['raw_text'].str.split('\n\n\n')


In [44]:
type(raw6.iloc[0].loc['raw_text'])

list

In [45]:
for item in raw6.iloc[0].loc['raw_text']:
    print(item)

REPORT CONTAMINACIÓN nº: 81587
FACILITY NAME: INEOS Infrastructure (Grangemouth) Ltd 81588
FacilityInspireID: UK.SEPA/200002494.Facility
COUNTRY: United Kingdom CONTINENT: EUROPE
CITY: Grangemouth
EPRTRSectorCode: 1 eprtrSectorName: Energy sector
MainActivityCode:1(c)
targetRealase: AIR pollutant: Carbon dioxide (CO2)
emissions: 4,29E+08
DAY: 10 MONTH: 8 YEAR: 2019
METEOROLOGICAL CONDITIONS
max_wind_speed: 1,72E+16 min_wind_speed: 2,36E+16 avg_wind_speed: 1,9E+15
max_temp: 6,52E+15 min_temp: 1,15E+16 avg_temp: 9,45E+15
DAYS FOG: 4
REPORTER NAME:Teresa Soto
CITY_ID ce6badb6a385e25ce519dc600eb7402e




In [46]:
data6_d = []
for my_list in raw6['raw_text']:
    data_dict = {}
    for item in my_list:
        try:
            data_dict.update({item.split(':', 1)[0].strip() : item.split(':', 1)[1].strip()} )
        except IndexError:
            if item[:6] == 'CITY_ID':
                data_dict.update({item.split()[0] : item.split()[1] })
            else: 
                continue
    data6_d.append(data_dict)
data6_d

def pdf_data_clean(the_data: dict) -> dict:
    pass

In [47]:
idx = 3
data6_d[idx]['FACILITY NAME'] , data6_d[idx]['FACILITY NAME'].strip(r'0123456789 ')

('Auchinlea Landfill Site, Motherwell 81536',
 'Auchinlea Landfill Site, Motherwell')

In [48]:
data6_d[2]

{'CITY': 'Grangemouth',
 'COUNTRY': 'United Kingdom CONTINENT: EUROPE',
 'DAY': '9 MONTH: 5 YEAR: 2019',
 'DAYS FOG': '3',
 'EPRTRSectorCode': '4 eprtrSectorName: Chemical industry',
 'FACILITY NAME': 'INEOS Chemicals Grangemouth Ltd 81585',
 'FacilityInspireID': 'UK.SEPA/200002493.Facility',
 'MainActivityCode': '4(a)(i)',
 'REPORT CONTAMINACIÓN nº': '81584',
 'REPORTER NAME': 'Kelly Williams',
 'emissions': '5,22E+08',
 'max_temp': '1,47E+16 min_temp: 1,78E+15 avg_temp: 1,66E+15',
 'max_wind_speed': '1,75E+16 min_wind_speed: 2,46E+15 avg_wind_speed: 1,8E+15',
 'targetRealase': 'AIR pollutant: Carbon dioxide (CO2)'}

In [49]:
for i in range(82):
  data6_d[i]['CONTINENT']= " ".join(data6_d[i]["COUNTRY"].split(" ", 3)[3:])
  data6_d[i]['MONTH']= " ".join(data6_d[i]["DAY"].split(" ", 4)[2:3])
  data6_d[i]['YEAR']= " ".join(data6_d[i]["DAY"].split(" ", 4)[4:])
  data6_d[i]['eprtrSectorName'] =" ".join(data6_d[i]["EPRTRSectorCode"].split(" ", 2)[2:])
  data6_d[i]['min_temp'] = " ".join(data6_d[i]["max_temp"].split(" ", 4)[2:3])
  data6_d[i]['avg_temp'] = " ".join(data6_d[i]["max_temp"].split(" ", 4)[4:])
  data6_d[i]['min_wind_speed'] = " ".join(data6_d[i]["max_wind_speed"].split(" ", 4)[2:3])
  data6_d[i]['avg_wind_speed'] = " ".join(data6_d[i]["max_wind_speed"].split(" ", 4)[4:])
  data6_d[i]['pollutant'] = " ".join(data6_d[i]["targetRealase"].split(" ", 2)[2:])

In [50]:
for i in range(82):
  data6_d[i]["EPRTRSectorCode"]= " ".join(data6_d[i]["EPRTRSectorCode"].split(" ", 1)[:1])
  data6_d[i]["max_wind_speed"]= " ".join(data6_d[i]["max_wind_speed"].split(" ", 1)[:1])
  data6_d[i]["max_temp"]=" ".join(data6_d[i]["max_temp"].split(" ", 1)[:1])
  data6_d[i]["targetRealase"]=" ".join(data6_d[i]["targetRealase"].split(" ", 1)[:1])
  data6_d[i]["COUNTRY"]=" ".join(data6_d[i]["COUNTRY"].split(" ", 3)[:2])
  data6_d[i]["DAY"]=" ".join(data6_d[i]["DAY"].split(" ", 1)[:1])

In [51]:
data6_d[1]['CONTINENT']

'EUROPE'

# **Trabajar con train6**

In [52]:
train6 = pd.DataFrame(data6_d)

In [53]:
train6.shape

(82, 23)

In [54]:
train6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   REPORT CONTAMINACIÓN nº  82 non-null     object
 1   FACILITY NAME            82 non-null     object
 2   FacilityInspireID        82 non-null     object
 3   COUNTRY                  82 non-null     object
 4   CITY                     82 non-null     object
 5   EPRTRSectorCode          82 non-null     object
 6   MainActivityCode         82 non-null     object
 7   targetRealase            82 non-null     object
 8   emissions                82 non-null     object
 9   DAY                      82 non-null     object
 10  max_wind_speed           82 non-null     object
 11  max_temp                 82 non-null     object
 12  DAYS FOG                 82 non-null     object
 13  REPORTER NAME            82 non-null     object
 14  CONTINENT                82 non-null     obj

In [55]:
train6.rename(columns = {'DAYS FOG':'DAY WITH FOGS', 'FACILITY NAME':'facilityName', "MainActivityCode":'EPRTRAnnexIMainActivityCode', 
                        'YEAR':'reportingYear', 'COUNTRY':'countryName', 'targetRealase': 'targetRelease', 'CITY': "City"}, inplace = True)

In [56]:
train6.columns

Index(['REPORT CONTAMINACIÓN nº', 'facilityName', 'FacilityInspireID',
       'countryName', 'City', 'EPRTRSectorCode', 'EPRTRAnnexIMainActivityCode',
       'targetRelease', 'emissions', 'DAY', 'max_wind_speed', 'max_temp',
       'DAY WITH FOGS', 'REPORTER NAME', 'CONTINENT', 'MONTH', 'reportingYear',
       'eprtrSectorName', 'min_temp', 'avg_temp', 'min_wind_speed',
       'avg_wind_speed', 'pollutant'],
      dtype='object')

In [57]:
train6.shape

(82, 23)

In [58]:
train6.drop('REPORT CONTAMINACIÓN nº', axis=1, inplace=True)

In [59]:
train6 = train6.merge(activity_code, on='EPRTRAnnexIMainActivityCode')

In [60]:
train6['max_wind_speed'] = train6['max_wind_speed'].apply(lambda x: x.replace(",", "."))

In [61]:
train6['max_wind_speed'] = train6['max_wind_speed'].astype(float)

In [62]:
train6['avg_wind_speed'] = train6['avg_wind_speed'].apply(lambda x: x.replace(",", "."))

In [63]:
train6['avg_wind_speed'] = train6['avg_wind_speed'].astype(float)

In [64]:
train6['min_wind_speed'] = train6['min_wind_speed'].apply(lambda x: x.replace(",", "."))

In [65]:
train6['min_wind_speed'] = train6['min_wind_speed'].astype(float)

In [66]:
train6['max_temp'] = train6['max_temp'].apply(lambda x: x.replace(",", "."))

In [67]:
train6['max_temp'] = train6['max_temp'].astype(float)

In [68]:
train6['avg_temp'] = train6['avg_temp'].apply(lambda x: x.replace(",", "."))

In [69]:
train6['avg_temp'] = train6['avg_temp'].astype(float)

In [70]:
train6['min_temp'] = train6['min_temp'].apply(lambda x: x.replace(",", "."))

In [71]:
train6['min_temp'] = train6['min_temp'].astype(float)

In [72]:
df1 = pd.concat([df1,train6], axis=0)

In [73]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 379078 entries, 0 to 350576
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   CITY ID                       28501 non-null   object
 1   CONTINENT                     379078 non-null  object
 2   City                          379078 non-null  object
 3   DAY                           379078 non-null  object
 4   DAY WITH FOGS                 379078 non-null  object
 5   EPRTRAnnexIMainActivityCode   379078 non-null  object
 6   EPRTRAnnexIMainActivityLabel  379078 non-null  object
 7   EPRTRSectorCode               379078 non-null  object
 8   FacilityInspireID             379078 non-null  object
 9   MONTH                         379078 non-null  object
 10  REPORTER NAME                 379078 non-null  object
 11  avg_temp                      379078 non-null  object
 12  avg_wind_speed                379078 non-null  object
 13 

In [74]:
df1 = df1.drop('emissions', axis=1)

In [75]:
df1.shape

(379078, 23)

In [76]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28559 entries, 0 to 9995
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   countryName                   28559 non-null  object 
 1   eprtrSectorName               28559 non-null  object 
 2   EPRTRAnnexIMainActivityLabel  28559 non-null  object 
 3   FacilityInspireID             28559 non-null  object 
 4   facilityName                  28559 non-null  object 
 5   City                          28559 non-null  object 
 6   targetRelease                 28559 non-null  object 
 7   pollutant                     28559 non-null  object 
 8   reportingYear                 28559 non-null  int64  
 9   MONTH                         28559 non-null  int64  
 10  DAY                           28559 non-null  int64  
 11  CONTINENT                     28559 non-null  object 
 12  max_wind_speed                28559 non-null  float64
 13  av

# **Unificar las columnas**



In [77]:
df = pd.concat([df1,df2], axis=0)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407637 entries, 0 to 9995
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   CITY ID                       57060 non-null   object
 1   CONTINENT                     407637 non-null  object
 2   City                          407637 non-null  object
 3   DAY                           407637 non-null  object
 4   DAY WITH FOGS                 407637 non-null  object
 5   EPRTRAnnexIMainActivityCode   379078 non-null  object
 6   EPRTRAnnexIMainActivityLabel  407637 non-null  object
 7   EPRTRSectorCode               379078 non-null  object
 8   FacilityInspireID             407637 non-null  object
 9   MONTH                         407637 non-null  object
 10  REPORTER NAME                 407637 non-null  object
 11  avg_temp                      407637 non-null  object
 12  avg_wind_speed                407637 non-null  object
 13  c

In [79]:
train0 = df

In [80]:
test0 = pd.read_csv('test_x.csv')

In [81]:
test0.head()

,test_index,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,City,targetRelease,reportingYear,MONTH,DAY,CONTINENT,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,CITY ID
0,0,Poland,3,Mineral industry,3(a),Underground mining and related operations,PL.MŚ/000002357.FACILITY,Polska Grupa Górnicza sp. z o.o. Oddział KWK R...,Rydułtowy,AIR,2017,8,16,EUROPE,14.08,14.86,18.47,10.28,11.38,13.48,1,Brittany Buck,826b1de9dad293ae3e4f9cbaf6cf3420
1,1,Luxembourg,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,LU.CAED/000012000.FACILITY,Sidec,Diekirch,AIR,2008,11,22,EUROPE,16.05,17.62,22.62,6.63,8.84,13.42,0,Lauren Fisher,ed30a6667b40ba0a66198b3173e7353f
2,2,Netherlands,1,Energy sector,1(c),Thermal power stations and other combustion in...,NL.EEA/212857.FACILITY,Nuon Power Generation BV (Eemshaven),Eemshaven,AIR,2014,9,19,EUROPE,13.65,15.54,17.82,5.67,8.40,11.28,2,Linda Thompson,78e1082c3cfef3bdf3554da8d6afcc34
3,3,Sweden,5,Waste and wastewater management,5(d),Landfills (excluding landfills of inert waste ...,SE.CAED/10013901.Facility,HÖGBYTORPS AVFALLSANLÄGGNING,BRO,AIR,2011,7,17,EUROPE,16.34,17.46,19.96,6.16,7.57,9.44,2,Bethany Mcmillan,27f959641950d381869d746d7d0e7d4e
4,4,Portugal,1,Energy sector,1(c),Thermal power stations and other combustion in...,PT.EEA/133926.FACILITY,SPCG - Sociedade Portuguesa de Co-Geração Eléc...,SETÚBAL,AIR,2010,6,23,EUROPE,21.52,20.53,21.62,10.96,11.55,12.62,2,Sarah Hoffman,1cb71655d9e0bd5cedb2320bf5fdd8f7


In [82]:
test0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24480 entries, 0 to 24479
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   test_index                    24480 non-null  int64  
 1   countryName                   24480 non-null  object 
 2   EPRTRSectorCode               24480 non-null  int64  
 3   eprtrSectorName               24480 non-null  object 
 4   EPRTRAnnexIMainActivityCode   24480 non-null  object 
 5   EPRTRAnnexIMainActivityLabel  24480 non-null  object 
 6   FacilityInspireID             24480 non-null  object 
 7   facilityName                  24480 non-null  object 
 8   City                          24480 non-null  object 
 9   targetRelease                 24480 non-null  object 
 10  reportingYear                 24480 non-null  int64  
 11  MONTH                         24480 non-null  int64  
 12  DAY                           24480 non-null  int64  
 13  C

In [83]:
target = train0['pollutant']

train1 = train0.drop(['CITY ID','REPORTER NAME','EPRTRSectorCode', 'EPRTRAnnexIMainActivityCode', 'FacilityInspireID'], axis=1)
test1 = test0.drop(['test_index', 'CITY ID', 'EPRTRSectorCode', 'EPRTRAnnexIMainActivityCode','FacilityInspireID', 'REPORTER NAME'], axis=1)

In [84]:
target.value_counts()

Methane (CH4)            223366
Nitrogen oxides (NOX)     97371
Carbon dioxide (CO2)      86900
Name: pollutant, dtype: int64

In [85]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407637 entries, 0 to 9995
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   CONTINENT                     407637 non-null  object
 1   City                          407637 non-null  object
 2   DAY                           407637 non-null  object
 3   DAY WITH FOGS                 407637 non-null  object
 4   EPRTRAnnexIMainActivityLabel  407637 non-null  object
 5   MONTH                         407637 non-null  object
 6   avg_temp                      407637 non-null  object
 7   avg_wind_speed                407637 non-null  object
 8   countryName                   407637 non-null  object
 9   eprtrSectorName               407637 non-null  object
 10  facilityName                  407637 non-null  object
 11  max_temp                      407637 non-null  object
 12  max_wind_speed                407637 non-null  object
 13  m

In [86]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24480 entries, 0 to 24479
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   countryName                   24480 non-null  object 
 1   eprtrSectorName               24480 non-null  object 
 2   EPRTRAnnexIMainActivityLabel  24480 non-null  object 
 3   facilityName                  24480 non-null  object 
 4   City                          24480 non-null  object 
 5   targetRelease                 24480 non-null  object 
 6   reportingYear                 24480 non-null  int64  
 7   MONTH                         24480 non-null  int64  
 8   DAY                           24480 non-null  int64  
 9   CONTINENT                     24480 non-null  object 
 10  max_wind_speed                24480 non-null  float64
 11  avg_wind_speed                24480 non-null  float64
 12  min_wind_speed                24480 non-null  float64
 13  m

In [87]:
data1 = pd.concat([train1, test1], axis=0).reset_index(drop=True)

In [88]:
target['pollutant'] = df['pollutant'].map({'Methane (CH4)': 0, 'Nitrogen oxides (NOX)': 1, 'Carbon dioxide (CO2)': 2 }) 

In [89]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432117 entries, 0 to 432116
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   CONTINENT                     432117 non-null  object
 1   City                          432117 non-null  object
 2   DAY                           432117 non-null  object
 3   DAY WITH FOGS                 432117 non-null  object
 4   EPRTRAnnexIMainActivityLabel  432117 non-null  object
 5   MONTH                         432117 non-null  object
 6   avg_temp                      432117 non-null  object
 7   avg_wind_speed                432117 non-null  object
 8   countryName                   432117 non-null  object
 9   eprtrSectorName               432117 non-null  object
 10  facilityName                  432117 non-null  object
 11  max_temp                      432117 non-null  object
 12  max_wind_speed                432117 non-null  object
 13 

### Imputar datos

In [90]:
data1.drop_duplicates(subset=None, keep='first', inplace=True)

In [91]:
data1.head()

,CONTINENT,City,DAY,DAY WITH FOGS,EPRTRAnnexIMainActivityLabel,MONTH,avg_temp,avg_wind_speed,countryName,eprtrSectorName,facilityName,max_temp,max_wind_speed,min_temp,min_wind_speed,pollutant,reportingYear,targetRelease
0,EUROPE,Europoort Rotterdam,2,1,Chemical installations for the production on a...,3,12.326234356831371,15.68645015581054,Netherlands,Chemical industry,Indorama Ventures Europe BV,13.256816011792559,11.019328717116156,14.696895445152332,20.899761591708206,Carbon dioxide (CO2),2020,AIR
1,EUROPE,RION,3,2,Installations for the production of cement cli...,2,5.114688475007264,18.396179670690906,Greece,Mineral industry,TITAN CEMENT S.A. - DREPANO PLANT,4.528859186447803,14.5123950384412,9.219003402711184,23.243402867192145,Nitrogen oxides (NOX),2019,AIR
2,EUROPE,--,1,12,Thermal power stations and other combustion in...,9,10.208837145825772,21.85621043510892,United Kingdom,Energy sector,Cormorant Alpha,10.669132597893881,20.26217117993502,14.715465115792192,23.956529199327292,Nitrogen oxides (NOX),2009,AIR
3,EUROPE,DRIZZONA,17,1,Installations for the intensive rearing of pou...,10,9.50230163010951,19.195731394748613,Italy,Intensive livestock production and aquaculture,SOCIETA' AGRICOLA SPARAVALLE DI FERRARI GIUSEP...,7.095681595088376,18.28354666681811,13.582024001859644,26.69626609353847,Methane (CH4),2014,AIR
4,EUROPE,Lünen,6,0,Installations for the recovery or disposal of ...,9,11.585549635691075,18.277089492642343,Germany,Waste and wastewater management,Biomassekraftwerk Lünen GmbH,9.886774464050356,13.75940846376134,14.00622637509683,24.768932565830674,Carbon dioxide (CO2),2015,AIR


In [92]:
data1['pollutant'] = data1['pollutant'].map({'Methane (CH4)': 0, 'Nitrogen oxides (NOX)': 1, 'Carbon dioxide (CO2)': 2 }) 

In [93]:
data1['DAY'] = data1['DAY'].astype(int)
data1['DAY WITH FOGS'] = data1['DAY WITH FOGS'].astype(int)
data1['MONTH'] = data1['MONTH'].astype(int)
data1['avg_temp'] = data1['avg_temp'].astype(float)
data1['avg_wind_speed'] = data1['avg_wind_speed'].astype(float)
data1['max_temp'] = data1['max_temp'].astype(float)
data1['max_wind_speed'] = data1['max_wind_speed'].astype(float)
data1['min_temp'] = data1['min_temp'].astype(float)
data1['min_wind_speed'] = data1['min_wind_speed'].astype(float)
data1['reportingYear'] = data1['reportingYear'].astype(int)

In [94]:
data1 = pd.get_dummies(data1, columns=['CONTINENT', 'City', 'EPRTRAnnexIMainActivityLabel', 'countryName', 'eprtrSectorName', 'facilityName', 'targetRelease'])

In [95]:
data1.shape

(81622, 14149)

In [96]:
data1 = data1.reset_index(drop=True)

In [97]:
train_final = data1.loc[:train0.index.max(), :].copy()
test_final = data1.loc[train0.index.max() + 1:, :].reset_index(drop=True).copy()

In [98]:
train_final.dropna(axis=0, inplace=True)

In [99]:
X = train_final.drop(['pollutant'], axis = 1)
y = train_final['pollutant']

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [101]:
y_train.value_counts()

1.00    18040
2.00    16013
0.00    11660
Name: pollutant, dtype: int64

In [102]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [103]:
pipelines = {
    'rf': make_pipeline(RandomForestClassifier(random_state=42)),
    'gb': make_pipeline(GradientBoostingClassifier(random_state=42)),
    'l1': make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', random_state= 1,solver='liblinear')),
    'l2': make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', random_state= 1,solver='liblinear')),
    'dt': make_pipeline(DecisionTreeClassifier(random_state=42)),
    'knn': make_pipeline(StandardScaler(), KNeighborsClassifier()),
    'svc':make_pipeline(StandardScaler(),SVC(random_state=42))
}

In [104]:
for key, value in pipelines.items():
    print(key, type(value))

rf <class 'sklearn.pipeline.Pipeline'>
gb <class 'sklearn.pipeline.Pipeline'>
l1 <class 'sklearn.pipeline.Pipeline'>
l2 <class 'sklearn.pipeline.Pipeline'>
dt <class 'sklearn.pipeline.Pipeline'>
knn <class 'sklearn.pipeline.Pipeline'>
svc <class 'sklearn.pipeline.Pipeline'>


In [105]:
rf_hyperparameters = {
    'randomforestclassifier__criterion':['entropy','gini'],
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_features': ['auto', 0.3, 0.6],
    'randomforestclassifier__min_samples_split':[2,3,10],
    'randomforestclassifier__min_samples_leaf':[1,3,10],
    'randomforestclassifier__bootstrap':[True, False]
}

gb_hyperparameters = {
    'gradientboostingclassifier__n_estimators':[100, 500, 100],
    'gradientboostingclassifier__learning_rate': [0.001,0.01,0.1,0.05],
    'gradientboostingclassifier__max_depth':[3, 5, 10]
}

l1_hyperparameters = {
    'logisticregression__C': np.logspace(-3,3,7)
}
l2_hyperparameters = {
    'logisticregression__C': np.logspace(-3,3,7)
}
dt_hyperparameters = {
    "decisiontreeclassifier__max_depth" : [1,3,5,7],
    'decisiontreeclassifier__criterion':['entropy','gini'],
    'decisiontreeclassifier__splitter':['best']
}

knn_hyperparameters = {
    "kneighborsclassifier__n_neighbors": np.arange(1,50),
    "kneighborsclassifier__weights": ["uniform","distance"]
}
svc_hyperparameters = {
    'svc__C': [1.0,], 
    'svc__kernel': ['rbf',], 
    'svc__degree': [2,], 
    'svc__gamma' : [1.0,], 
    'svc__shrinking':[True,], 
    'svc__probability':[True,],
    'svc__tol': [0.001,],
    'svc__cache_size':[200,]
}


In [106]:
hyperparameter_grids = {
    'rf': rf_hyperparameters,
    'gb': gb_hyperparameters,
    'l1':l1_hyperparameters,
    'l2':l2_hyperparameters,
    'dt':dt_hyperparameters,
    'knn': knn_hyperparameters,
    'svc':svc_hyperparameters
}

In [107]:
for key in ['rf', 'gb', 'l1', 'l2', 'dt', 'knn', 'svc']:
    if key in hyperparameter_grids:
        if type(hyperparameter_grids[key]) is dict:
            print( key, 'was found, and it is a grid.' )
        else:
            print( key, 'was found, but it is not a grid.' )
    else:
        print( key, 'was not found')

rf was found, and it is a grid.
gb was found, and it is a grid.
l1 was found, and it is a grid.
l2 was found, and it is a grid.
dt was found, and it is a grid.
knn was found, and it is a grid.
svc was found, and it is a grid.


In [110]:
models = {}

for key in pipelines.keys():
    models[key]= GridSearchCV(pipelines[key], hyperparameter_grids[key], cv=5, scoring = 'f1_macro',
                             n_jobs= -1,
                             verbose = 1)
    
models.keys()

dict_keys(['rf', 'gb', 'l1', 'l2', 'dt', 'knn', 'svc'])

In [ ]:
best_estimators = []

for key in models.keys():
    clf = models[key].fit(X_res,y_res)
    best_estimators.append(clf.best_estimator_)
    print(key, "is trained and tuned")
    print('Method: {} Best: {}' .format(key,clf.best_estimator_))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


In [ ]:
for key in models.keys() :
    pred = models[key].predict(X_test)
    print(key)
    print("Score del modelo (accuracy):", round(models[key].score(X_test, y_test), 3))
    print("Accuracy score:", round(accuracy_score(pred, y_test), 3))
    print("Recall score:", round(recall_score(pred, y_test, average='macro'), 3))
    print("Precision score:", round(precision_score(pred, y_test, average='macro'), 3))
    print("F1 score:", round(f1_score(pred, y_test, average='macro'), 3))
    print('---')

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
estimator_list = [('rf',best_estimators[0]),
                                        ('gb',best_estimators[1]),
                                        ('l1',best_estimators[2]),
                                        ('l2',best_estimators[3]),
                                        ('dt',best_estimators[4]),
                                        ('knn', best_estimators[5]),
                                        ('svc', best_estimators[6])
                                       ]

In [ ]:
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=MLPClassifier()
)


In [ ]:
stacking_c = stack_model.fit(X_res,y_res)
print("Score del modelo (accuracy):", round(stacking_c.score(X_test, y_test), 3))
print("Accuracy score:", round(accuracy_score(stacking_c.predict(X_test), y_test), 3))
print("Recall score:", recall_score(stacking_c.predict(X_test), y_test, 
                                           average='macro'))
print("Precision score:", precision_score(stacking_c.predict(X_test), y_test, average='macro'))
print("F1 score:", f1_score(stacking_c.predict(X_test), y_test, average='macro'))

In [ ]:
test_final['pollutant']= stacking_c.predict(test_final)

In [ ]:
test_final.head()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58d524ea-a1f6-4e66-a18d-4bc099b95b9f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>